# Zip Code Census Data

## Importing

In [ ]:
import pandas as pd

In [ ]:
big = pd.read_csv('ACSST5Y2020.S2701-Data.csv')

## Cleaning Census Data

In [ ]:
#Change the column names to the first row of data
big.rename(columns = big.loc[0], inplace=True)
big.drop(0, axis = 0, inplace=True)

In [ ]:
#Changing the zip code values to only include the zip code and not 'ZCTA5'
big['Geographic Area Name'] = big['Geographic Area Name'].str.replace('ZCTA5 ', '')

In [ ]:
#Creating a smaller dataframe that only includes total population, insured population, and uninsured population
small = big[['Geographic Area Name','Estimate!!Total!!Civilian noninstitutionalized population','Estimate!!Insured!!Civilian noninstitutionalized population','Estimate!!Uninsured!!Civilian noninstitutionalized population']]

In [ ]:
#Renaming the new dataframe's columns
small.columns = ['zip code','population','insured','uninsured']

In [ ]:
#Changing the columns to numeric types
small['population'] = pd.to_numeric(small['population'])
small['insured'] = pd.to_numeric(small['insured'])
small['uninsured'] = pd.to_numeric(small['uninsured'])

In [ ]:
#Creating two percentage columns for insured and uninsured
small = small.assign(percent_insured=small['insured'] / small['population']*100)
small = small.assign(percent_uninsured=small['uninsured'] / small['population']*100)

In [ ]:
#Rounding the columns to 2 decimal places
small['percent_insured']=small['percent_insured'].round(2)
small['percent_uninsured']=small['percent_uninsured'].round(2)

## Making the heatmap

### Importing

In [ ]:
import geopandas as gdp
import matplotlib.pyplot as plt

#### Zip Code Shape

In [ ]:
#Create the us zip code shape dataframe
us_zip = gdp.read_file('tl_2022_us_zcta520.shp')

In [ ]:
#Creating a tennessee zip code list using the small dataframe
zips = small['zip code'].tolist()

In [ ]:
#Filtering the shape file to only include tennessee zip codes
tn_zip = us_zip[us_zip['ZCTA5CE20'].isin(zips)]

In [ ]:
#Changing the tennessee zip code column to make it easy to merge
tn_zip = tn_zip.rename(columns={'ZCTA5CE20':'zip code'})

In [ ]:
#Merge the shape dataframe to the small small dataframe
final = tn_zip.merge(small, on='zip code')

#### County Shape

In [ ]:
#Importing the county shape file
us_shape = gdp.read_file('cb_2022_us_county_500k.shp')

In [ ]:
#Filtering the shape file to only include TN
tn_shape = us_shape[us_shape['STUSPS']=='TN']

### Entire Tennessee Zip Code Choropleth

In [ ]:
#Creating figure
fig, ax = plt.subplots(1, figsize=(10,10))
plt.xticks(rotation=90)

final.plot(column='percent_uninsured', cmap='Reds', linewidth=0.4, ax=ax, edgecolor=".4")

### Creating a Choropleth overlay

In [ ]:
#Create figure
fig, ax = plt.subplots(figsize=(10, 10))

#Plot choropleth
final.plot(column='population', cmap='Reds', linewidth=0.8, ax=ax, edgecolor='0.8')

#Overlay counties
tn_shape.plot(ax=ax, linewidth=1, edgecolor='Black', facecolor='none')

#Title and axis
ax.set_title('Choropleth Map with Additional Shape Outline')
ax.set_axis_off()

#Show the plot
plt.show()


### Creating Specific Area Choropleth

In [ ]:
#Creating list with specific county cip codes
czip =['37018','37342','37355','37360','37388',
       '37389','37306','37318','37324','37330',
       '37345','37375','37376','37398','37352']

In [ ]:
#Filtering the zip code shape file
county = tn_zip[tn_zip['zip code'].isin(czip)]

In [ ]:
#Merge the county shape file on the small dataframe
cfinal = county.merge(small, on='zip code')

#### Creating smaller zip code choropleth

In [ ]:
#Create figure
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)

#Choropleth
cfinal.plot(column='percent_uninsured', cmap='Reds', linewidth=0.8, ax=ax, edgecolor='0.8')

#Title and remove axis
ax.set_title('Percent Uninsured by Population')
ax.set_axis_off()

#Add scale bar
bar = plt.cm.ScalarMappable(cmap='Reds', norm=plt.Normalize(vmin=0, vmax='25'))
bar._A = []
cbar = fig.colorbar(bar, shrink=0.75)

#Show the plot
plt.show()

### Creating a specific area choropleth with overlay

In [ ]:
#Creating a list of the counties that the clinic serves
pfh = ['Franklin', 'Coffee', 'Moore']

In [ ]:
#Filtering the county shape file to include these three counties
tri = tn_shape[tn_shape['NAME'].isin(pfh)]

In [ ]:
#Create a new figure and axis
fig, ax = plt.subplots(figsize=(10, 10))

#Plot the choropleth map
cfinal.plot(column='percent_uninsured', cmap='Reds', linewidth=0.8, ax=ax, edgecolor='0.8')

#Background
fig.patch.set_facecolor('lightgray')

#Overlay additional shape(s) as outline
tri.plot(ax=ax, linewidth=1, edgecolor='black', facecolor='none')

#Customize the plot
ax.set_title('Percent Uninsured by Population')
ax.set_axis_off()


#Add scale bar
bar = plt.cm.ScalarMappable(cmap='Reds', norm=plt.Normalize(vmin=0, vmax='25'))
bar._A = []
cbar = fig.colorbar(bar)

#Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt

#Create a new figure and a grid of subplots with adjusted height ratios
fig, axes = plt.subplots(2, 1, figsize=(10, 12), gridspec_kw={'height_ratios': [3, 1]})

#Set background
fig.patch.set_facecolor('lightgrey')

#Plot the choropleth
ax_choropleth = axes[0]
choropleth = cfinal.plot(column='percent_uninsured', cmap='Reds', linewidth=0.8, ax=ax_choropleth, edgecolor='0.8')
ax_choropleth.set_title('Percent Uninsured by Population')
ax_choropleth.set_axis_off()

#Add scale bar
bar = plt.cm.ScalarMappable(cmap='Reds', norm=plt.Normalize(vmin=0, vmax=25))
bar._A = []
cbar = fig.colorbar(bar, ax=ax_choropleth)

#Overlay counties
tri.plot(ax=ax_choropleth, linewidth=1, edgecolor='black', facecolor='none')

#Get the top 10 zip codes by percentage uninsured
ctop_10_zipcodes = cfinal.nlargest(10, 'percent_uninsured')

#Get choropleth colors
colors = [bar.to_rgba(val) for val in ctop_10_zipcodes['percent_uninsured']]

#Plot the bar chart
ax_bar = axes[1]
ax_bar.bar(ctop_10_zipcodes['zip code'], ctop_10_zipcodes['percent_uninsured'], color=colors)
ax_bar.set_title('Top 10 Zip Codes by Percentage Uninsured')
ax_bar.set_xlabel('Zip Code')
ax_bar.set_ylabel('Percentage')
ax_bar


In [ ]:
import matplotlib.pyplot as plt

#Create a new figure and a grid of subplots with adjusted height ratios
fig, axes = plt.subplots(2, 1, figsize=(10, 12), gridspec_kw={'height_ratios': [3, 1]})

#Set the background color to grey for the entire figure
fig.patch.set_facecolor('lightgrey')

#Plot the choropleth
ax_choropleth = axes[0]
choropleth = cfinal.plot(column='percent_insured', cmap='Blues', linewidth=0.8, ax=ax_choropleth, edgecolor='0.8')
ax_choropleth.set_title('Percent Insured by Population')
ax_choropleth.set_axis_off()

#Add scale bar to the choropleth map
bar = plt.cm.ScalarMappable(cmap='Blues', norm=plt.Normalize(vmin=0, vmax=100))
bar._A = []
cbar = fig.colorbar(bar, ax=ax_choropleth)

#Overlay counties
tri.plot(ax=ax_choropleth, linewidth=1, edgecolor='black', facecolor='none')

#Get the top 10 zip codes by percentage uninsured
ctop_10_zipcodes = cfinal.nlargest(10, 'percent_insured')

#Get choropleth colors
colors = [bar.to_rgba(val) for val in ctop_10_zipcodes['percent_insured']]

#Plot the bar chart
ax_bar = axes[1]
ax_bar.bar(ctop_10_zipcodes['zip code'], ctop_10_zipcodes['percent_insured'], color=colors)
ax_bar.set_title('Top 10 Zip Codes by Percentage Insured')
ax_bar.set_xlabel('Zip Code')
ax_bar.set_ylabel('Percentage')
ax_bar


### Creating an adjacent county choropleth with overlay

In [ ]:
#Creating zip code list of adjacent counties
aa = ['37018','37342','37355','37360','37388','37389','37306','37318',
      '37324','37330','37345','37375','37376','37398','37352','37301',
      '37305','37313','37339','37356','37365','37366','37387','37110',
      '37357','38581','37020','37160','37180','37183','37016','37026',
      '37149','37190']

In [ ]:
#Creating a shape file for adjacent county zip codes
adjcounty = tn_zip[tn_zip['zip code'].isin(aa)]

In [ ]:
#Merging the adjcounty shape file with the census data
adjfinal = adjcounty.merge(small, on='zip code')

In [ ]:
#Creating list of adjacent counties
apfh = ['Franklin', 'Coffee', 'Moore', 'Warren', 'Grundy', 'Cannon', 'Bedford']

In [ ]:
#Creating adj county shape file
adj = tn_shape[tn_shape['NAME'].isin(apfh)]

In [ ]:
#Create a new figure and axis
fig, ax = plt.subplots(figsize=(10, 10))

#Plot choropleth
adjfinal.plot(column='percent_uninsured', cmap='Reds', linewidth=0.8, ax=ax, edgecolor='0.8')

#background
fig.patch.set_facecolor('lightgray')

#Overlay counties
adj.plot(ax=ax, linewidth=1, edgecolor='black', facecolor='none')

#Customize the plot
ax.set_title('Percent Uninsured by Population')
ax.set_axis_off()


#Add scale bar
bar = plt.cm.ScalarMappable(cmap='Reds', norm=plt.Normalize(vmin=0, vmax='25'))
bar._A = []
cbar = fig.colorbar(bar)

#Show the plot
plt.show()


In [ ]:
#Create a new figure and a grid of subplots with adjusted height ratios
fig, axes = plt.subplots(2, 1, figsize=(10, 12), gridspec_kw={'height_ratios': [3, 1]})

#Set background
fig.patch.set_facecolor('lightgrey')

#Plot the choropleth
ax_choropleth = axes[0]
choropleth = adjfinal.plot(column='percent_uninsured', cmap='Reds', linewidth=0.8, ax=ax_choropleth, edgecolor='0.8')
ax_choropleth.set_title('Percent Uninsured by Population')
ax_choropleth.set_axis_off()

#Add scale bar
bar = plt.cm.ScalarMappable(cmap='Reds', norm=plt.Normalize(vmin=0, vmax=25))
bar._A = []
cbar = fig.colorbar(bar, ax=ax_choropleth)

#Overlay counties
adj.plot(ax=ax_choropleth, linewidth=1, edgecolor='black', facecolor='none')

#Get the top 10 zip codes by percentage uninsured
top_10_zipcodes = adjfinal.nlargest(10, 'percent_uninsured')

#Get choropleth colors
colors = [bar.to_rgba(val) for val in top_10_zipcodes['percent_uninsured']]

#Plot the bar chart
ax_bar = axes[1]
ax_bar.bar(top_10_zipcodes['zip code'], top_10_zipcodes['percent_uninsured'], color=colors)
ax_bar.set_title('Top 10 Zip Codes by Percentage Uninsured')
ax_bar.set_xlabel('Zip Code')
ax_bar.set_ylabel('Percentage')
ax_bar


In [ ]:
#Create a new figure and a grid of subplots with adjusted height ratios
fig, axes = plt.subplots(2, 1, figsize=(10, 12), gridspec_kw={'height_ratios': [3, 1]})

#Set background
fig.patch.set_facecolor('lightgrey')

#Plot the choropleth
ax_choropleth = axes[0]
choropleth = adjfinal.plot(column='percent_uninsured', cmap='Reds', linewidth=0.8, ax=ax_choropleth, edgecolor='0.8')
ax_choropleth.set_title('Percent Uninsured by Population')
ax_choropleth.set_axis_off()

#Add scale bar to the choropleth map
bar = plt.cm.ScalarMappable(cmap='Reds', norm=plt.Normalize(vmin=0, vmax=25))
bar._A = []
cbar = fig.colorbar(bar, ax=ax_choropleth)

#Overlay counties
adj.plot(ax=ax_choropleth, linewidth=1, edgecolor='black', facecolor='none')

#Get the top 10 zip codes by percentage uninsured
top_10_zipcodes = adjfinal.nlargest(10, 'percent_uninsured')

#Get choropleth colors
colors = [bar.to_rgba(val) for val in top_10_zipcodes['percent_uninsured']]

#Plot the bar chart
ax_bar = axes[1]
ax_bar.bar(top_10_zipcodes['zip code'], top_10_zipcodes['percent_uninsured'], color=colors)
ax_bar.set_title('Top 10 Zip Codes by Percentage Uninsured')
ax_bar.set_xlabel('Zip Code')
ax_bar.set_ylabel('Percentage')
ax_bar


In [ ]:
#Create a new figure and a grid of subplots with adjusted height ratios
fig, axes = plt.subplots(2, 1, figsize=(10, 12), gridspec_kw={'height_ratios': [3, 1]})

#Set background
fig.patch.set_facecolor('lightgrey')

#Plot the choropleth
ax_choropleth = axes[0]
choropleth = adjfinal.plot(column='percent_insured', cmap='Blues', linewidth=0.8, ax=ax_choropleth, edgecolor='0.8')
ax_choropleth.set_title('Percent Insured by Population')
ax_choropleth.set_axis_off()

#Add scale bar
bar = plt.cm.ScalarMappable(cmap='Blues', norm=plt.Normalize(vmin=0, vmax=100))
bar._A = []
cbar = fig.colorbar(bar, ax=ax_choropleth)

#Overlay counties
adj.plot(ax=ax_choropleth, linewidth=1, edgecolor='black', facecolor='none')

#Get the top 10 zip codes by percentage uninsured
top_10_zipcodes = adjfinal.nlargest(10, 'percent_insured')

#Get choropleth colors
colors = [bar.to_rgba(val) for val in top_10_zipcodes['percent_insured']]

#Plot the bar chart
ax_bar = axes[1]
ax_bar.bar(top_10_zipcodes['zip code'], top_10_zipcodes['percent_insured'], color=colors)
ax_bar.set_title('Top 10 Zip Codes by Percentage Insured')
ax_bar.set_xlabel('Zip Code')
ax_bar.set_ylabel('Percentage')
ax_bar
